In [2]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Segmentação dos dados
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Modelagem
import tensorflow as tf
from keras.layers import LSTM,Dense,Bidirectional,Input
from tensorflow.keras.optimizers import Adam
import transformers
from tokenizers import BertWordPieceTokenizer
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import torch

ImportError: cannot import name 'tf2' from 'tensorflow.python' (unknown location)

# Importação dos dados

In [1]:
df = pd.read_csv('..\datasets\mouse_ajust.csv')

NameError: name 'pd' is not defined

In [5]:
df.head()

,Unnamed: 0,score,description,date
0,0,0,O produto aparece usado...em baixo do aparelho...,13 de novembro de 2019
1,1,2,"Mouse sem fio ótimo, sem delay, nano receptor ...",19 de agosto de 2019
2,2,1,"A construção do mouse em si é boa, os clicks s...",15 de março de 2019
3,3,2,A mídia não pôde ser carregada.\r\n ...,31 de julho de 2020
4,4,0,"O mouse está em boas condições, funcionando be...",21 de junho de 2020


In [113]:
# Remoção de uma classe
df['score'].replace({1:0},inplace=True)

In [114]:
df['score'].replace({2:1},inplace=True)

# Train-test-split [Desbalanceado]

In [116]:
y = df['score']
x = df.iloc[:,1]

#divisão do treino e teste

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
#Balanceamento 

'''ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(np.array(X_train).reshape(-1, 1), np.array(y_train).reshape(-1, 1))

print(X_resampled.shape)
print(y_resampled.shape)
print('\n',pd.DataFrame(y_resampled).value_counts())

X_train , y_train = pd.DataFrame(X_resampled), y_resampled

X_train = np.array(X_train[0])'''

# BERT com modelo pré-treinado

- Carregando o tokenizador pré-treinado pt-br

In [43]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased' , lower = True)

In [44]:
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer.enable_padding()

In [46]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=400):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

- Tokenizando test e treino

In [117]:
X_train = fast_encode(X_train.values, fast_tokenizer, maxlen=400)
X_test = fast_encode(X_test.values, fast_tokenizer, maxlen=400)

In [118]:
print(X_train.shape)
print(X_test.shape)

(3840, 400)
(960, 400)


In [14]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFDistilBertModel

In [58]:
bert_model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased',from_pt=True)

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [119]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [120]:
bert_model.compile(optimizer='adam', loss = loss, metrics=['accuracy'])

In [121]:
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108923136 
                                                                 
 dropout_56 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,924,674
Trainable params: 108,924,674
Non-trainable params: 0
_________________________________________________________________


In [122]:
history = bert_model.fit(X_train,y_train,batch_size = 10 ,validation_data=(X_test,y_test),epochs = 1)

384/384 [==============================] - 6215s 16s/step - loss: 0.2535 - accuracy: 0.9378 - val_loss: 0.2363 - val_accuracy: 0.9458


In [123]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

30/30 [==============================] - 177s 6s/step - loss: 0.7427 - accuracy: 0.9719
Accuracy of the model on Testing Data is -  97.1875011920929 %


In [136]:
# Salvando o modelo

export_dir='../models/bert_modelo'
tf.saved_model.save(bert_model, export_dir=export_dir)

INFO:tensorflow:Assets written to: ../models/bert_modelo\assets


INFO:tensorflow:Assets written to: ../models/bert_modelo\assets
